# Tidy Data

The aim of this Capstone project is to consolidate the knowledge, skills and competences acquired by learners during this programme. In this project students will select, specify, design, justify and implement a data analytics investigation that makes use of large data sets. The project will necessarily require data ingestion, wrangling, computation and analyses, and the production and presentation of a project and report. Students are guided by faculty in the selection of a data set, scope of investigation, initial exploratory data analysis, and formulation of hypotheses.

In [43]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ast
import re

In [44]:
deck_df = pd.read_csv("data/winning_decks.csv")

In [45]:
card_df = pd.read_csv("data/cards.csv")

There are 6 houses in total: Brobnar, Dis, Sanctum, Mars, Untamed, Shadows, Logos. We want to add those 6 columns and a boolean True of False for whether the deck has that house or not.

In [46]:
deck_df['house_brobnar'] = deck_df['houses'].str.contains('Brobnar')
deck_df['house_dis'] = deck_df['houses'].str.contains('Dis')
deck_df['house_sanctum'] = deck_df['houses'].str.contains('Sanctum')
deck_df['house_mars'] = deck_df['houses'].str.contains('Mars')
deck_df['house_untamed'] = deck_df['houses'].str.contains('Untamed')
deck_df['house_shadows'] = deck_df['houses'].str.contains('Shadows')
deck_df['house_logos'] = deck_df['houses'].str.contains('Logos')

In [47]:
deck_df = deck_df.drop("houses", axis=1)
deck_df.drop(deck_df.columns[deck_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

In [48]:
deck_df.head()

,deck_id,deck_name,deck_wins,deck_losses,deck_expansion,deck_list,house_brobnar,house_dis,house_sanctum,house_mars,house_untamed,house_shadows,house_logos
0,6216e565-a790-4e5b-b89a-4a814917b94e,"Leif “As”, Granjera de la Alcantarilla",2,1,435,"['417b7845-2c67-4bcf-b411-20f2a7d86955', '920f...",True,True,True,False,False,False,False
1,50e28cdd-6e6b-4584-a8d1-1f22c081e906,"“Ivan” Valencia, Ship Chieftain",1,3,435,"['2ee769bb-b785-48ea-a5fa-a19a2af32bbc', '2ee7...",True,False,True,False,True,False,False
2,3969e139-05d7-4a8a-ba60-0bb660cd057b,"Nasir, Canal Survivalist",2,1,435,"['a3234c40-8226-439d-bb8f-207ea26c9560', '920f...",True,False,True,True,False,False,False
3,5668bdc3-dea4-4226-92ba-91b003a5460e,"Destrolack, Observatory Recluse",4,4,435,"['7bf05cd5-7c03-4e06-9ace-bb730ff6100e', '7bf0...",False,False,True,False,True,False,True
4,749780f6-7acb-402a-9fd6-f6fbbcf888e1,"Tiberius, Navigatore della Guglia Aperta",8,4,341,"['753bfb51-4ba7-4c0a-b141-a5b6388498c0', 'aeed...",False,False,False,True,True,True,False


Next we need to look at the cards.

In [49]:
card_df.head()

,Unnamed: 0,Unnamed: 0.1,card_id,card_title,card_type,card_amber,card_power,card_armor,card_traits
0,0,0,bfaef0df-911c-4f21-a452-83d1144dc0c4,Ixxyxli Fixfinger,Creature,0,2.0,2.0,Martian • Scientist
1,1,1,e3a8d05a-252a-4d07-bebe-8a5b69efb98c,Yantzee Gang,Creature,0,5.0,NaN,Elf • Thief
2,2,2,98e9c095-d134-4bed-b6db-4bfacb433473,Scientifical Hack,Artifact,1,NaN,NaN,Equation
3,3,3,30dfd43a-4973-482a-8a52-be37d8cf0ca8,Cull the Weak,Action,1,NaN,NaN,NaN
4,4,4,38b9c734-5286-4c94-b92a-28e3aa4f1357,Oubliette,Action,0,NaN,NaN,NaN


For some cards they only appear once.

In [50]:
card_df.loc[card_df['card_title']=='Ixxyxli Fixfinger']

,Unnamed: 0,Unnamed: 0.1,card_id,card_title,card_type,card_amber,card_power,card_armor,card_traits
0,0,0,bfaef0df-911c-4f21-a452-83d1144dc0c4,Ixxyxli Fixfinger,Creature,0,2.0,2.0,Martian • Scientist


Other cards however appear under multiple card_ids, this is because they have special versions that appear in other houses. So if we look at the card 'Bait and Switch' we can see it appears a total of 7 times, one for each house.

In [51]:
card_df.loc[card_df['card_title']=='Bait and Switch']

,Unnamed: 0,Unnamed: 0.1,card_id,card_title,card_type,card_amber,card_power,card_armor,card_traits
448,797,182,0186f94c-68df-4d5c-9338-9e918affe313,Bait and Switch,Action,0,0.0,0.0,NaN
1005,95391,13,771eb74c-d367-46af-be2f-994cd78f0e8e,Bait and Switch,Action,0,0.0,0.0,NaN
1313,248243,286,a20770c0-dea0-4690-8ef6-c93e64105229,Bait and Switch,Action,0,0.0,0.0,NaN
2015,47598,171,771c498c-33bc-40b7-8d41-54fd0a7c2567,Bait and Switch,Action,0,0.0,0.0,NaN
2116,102887,211,0cf698fb-1d7d-4ecb-804a-fa7904501945,Bait and Switch,Action,0,0.0,0.0,NaN
2191,144447,22,ef9a58d8-9684-4f39-9a1d-cc93a04d0027,Bait and Switch,Action,0,0.0,0.0,NaN
2300,228373,60,bf82eb92-cfae-4b59-b676-f8f3013e4572,Bait and Switch,Action,0,0.0,0.0,NaN


If we want to add a boolean value for each card, we a column for each of the unique cards name which are a total of:

In [52]:
len(card_df.groupby('card_title')['card_title'].nunique())

574

So we need to add 574 columns. Lets start by getting the list of names.

In [53]:
series_of_cards = card_df['card_title']
list_of_cards = list(dict.fromkeys(series_of_cards))
list_of_cards.sort()
list_of_cards

['1-2 Punch',
 'A Fair Game',
 'Abond the Armorsmith',
 'Agent Hoo-man',
 'Ammonia Clouds',
 'Anahita the Trader',
 'Ancient Bear',
 'Ancient Yurk',
 'Anger',
 'Angwish',
 'Annihilation Ritual ',
 'Anomaly Exploiter',
 'Archimedes',
 'Arise!',
 'Armageddon Cloak',
 'Aubade the Grim',
 'Autocannon',
 'Backup Copy',
 'Bad Penny',
 'Bait and Switch',
 'Banish',
 'Banner of Battle',
 'Barehanded',
 'Baron Mengevin',
 'Barrister Joya',
 'Batdrone',
 'Battle Fleet',
 'Bear Flute',
 'Begone!',
 'Bellowing Patrizate',
 'Bigtwig',
 'Bilgum Avalanche',
 'Binate Rupture',
 'Binding Irons',
 'Bingle Bangbang',
 'Biomatrix Backup',
 'Blinding Light',
 'Blood Money',
 'Blood of Titans',
 'Bloodshard Imp',
 'Blypyp',
 'Bonerot Venom',
 'Booby Trap',
 'Bordan the Redeemed',
 'Bouncing Deathquark',
 'Brain Eater',
 'Brain Stem Antenna',
 'Brammo',
 'Brend the Fanatic',
 'Briar Grubbling',
 'Brobnar Ambassador',
 'Bronze Key Imp',
 'Brothers in Battle',
 'Bulleteye',
 'Bulwark',
 'Bumblebird',
 'Bumpsy'

We would like to have a True False check to see if the card is in the deck. This is problematic as there are multiple card_ids for each card name. That would be tricky to match. Instead what we'll try do, is add a new column which has the card names instead of the card ids.

In [54]:
def change_card_ids_to_names(card_list):
    new_card_list = []
    start=2
    end=38
    for i in range(36):
        card = card_df.loc[(card_df['card_id'] == \
                           card_list[start:end])]['card_title'].item()
        new_card_list.append(card)
        start = start + 40
        end = end + 40
    return(new_card_list)

In [55]:
deck_df.iloc[0]['deck_list']

"['417b7845-2c67-4bcf-b411-20f2a7d86955', '920f3001-2f7a-4c21-b5b6-5d95fc5c0fef', '0ec85771-8143-4d64-8c3a-ce48bce4e81c', '4e6ddfb1-80bb-408c-9470-9a17120b96dc', 'eb3bd495-1d1a-45f6-941e-a0cf159431d1', 'bf88c08c-9cb3-4541-8330-5f209104c7d5', '3293f116-662c-4484-8951-accb4f25dd00', '69252efa-9686-4868-aca9-b8bc5af7cb8a', 'ef95c09f-7134-470c-8ddf-7ae1c84bbcb0', '52e8b81a-a716-4147-afcf-560bec2a3523', '16cdef01-a9d6-44d9-b0e0-247544e1db7f', 'b822fd9d-8209-49dd-b1b1-541f54641bf1', '0c75c0ba-d9ac-48f0-a1a2-72ae29000ea2', '17387482-a223-488a-a8ba-6911d691c281', '17387482-a223-488a-a8ba-6911d691c281', '9b5ad855-50f3-43e6-ad85-af95da564469', '29ddc45f-882b-4cb2-bb9b-efb91bd09e82', '7dcc8edb-ee57-4152-9b5d-cc012fa9973b', '7dcc8edb-ee57-4152-9b5d-cc012fa9973b', '7dcc8edb-ee57-4152-9b5d-cc012fa9973b', 'dc2f26d0-a6ab-4e15-b102-f23d6d26969b', '300dda74-ff5c-42e8-b17a-ff4eebc8109f', '77a3d587-3590-4fd3-80a1-1961f7c74ac7', 'dec37837-bdf2-4389-8204-92b75d32c9eb', 'eeac7978-6fa5-4c6a-a62e-144692ba93d6'

In [56]:
change_card_ids_to_names(deck_df.iloc[0]['deck_list'])

['Bingle Bangbang',
 'Cowfyne',
 'First Blood',
 'Foozle',
 'Ganger Chieftain',
 'Gauntlet of Command',
 'Pound',
 'Blood Money',
 'Blood of Titans',
 'Sound the Horns',
 'The Flex',
 'Tireless Crocag',
 'Lash of Broken Dreams',
 'Not Finished with You',
 'Not Finished with You',
 'Shooler',
 'Streke',
 'Yurk',
 'Yurk',
 'Yurk',
 'Gongoozle',
 'Pain Reaction',
 'Pandemonium',
 'Tentacus',
 'Bordan the Redeemed',
 'Challe the Safeguard',
 'Challe the Safeguard',
 'Golden Aura',
 'Hallowed Shield',
 'Maruck the Marked',
 'Protect the Weak',
 'Smite',
 'The Grey Rider',
 'Potion of Invulnerability',
 'Duma the Martyr',
 'Eye of Judgment']

This next bit takes some time.

In [57]:
deck_df['card_name_list'] = deck_df['deck_list'].apply(change_card_ids_to_names)

In [58]:
deck_df.iloc[0]

deck_id                        6216e565-a790-4e5b-b89a-4a814917b94e
deck_name                    Leif “As”, Granjera de la Alcantarilla
deck_wins                                                         2
deck_losses                                                       1
deck_expansion                                                  435
deck_list         ['417b7845-2c67-4bcf-b411-20f2a7d86955', '920f...
house_brobnar                                                  True
house_dis                                                      True
house_sanctum                                                  True
house_mars                                                    False
house_untamed                                                 False
house_shadows                                                 False
house_logos                                                   False
card_name_list    [Bingle Bangbang, Cowfyne, First Blood, Foozle...
Name: 0, dtype: object

The card name [REDACTED] causes problems later, so we rename to avoid this.

In [59]:
list_of_cards[568] = "REDACTED"

Now we can use a similar method as we did with the houses using a check to see if the name is included in the field. This cycles through the list of 574 card columns and counts the number of times the card name appears in the total card list string.

In [60]:
for n in list_of_cards:
    deck_df[n] = deck_df['card_name_list'].astype(str).str.count(n)

In [61]:
deck_df.iloc[0]

deck_id                               6216e565-a790-4e5b-b89a-4a814917b94e
deck_name                           Leif “As”, Granjera de la Alcantarilla
deck_wins                                                                2
deck_losses                                                              1
deck_expansion                                                         435
deck_list                ['417b7845-2c67-4bcf-b411-20f2a7d86955', '920f...
house_brobnar                                                         True
house_dis                                                             True
house_sanctum                                                         True
house_mars                                                           False
house_untamed                                                        False
house_shadows                                                        False
house_logos                                                          False
card_name_list           

Next we want to assign a 'score' to each deck, this will be our target variable. In this case we are going to take the number of wins and subtract the number of losses.

In [62]:
deck_df.describe()

,deck_wins,deck_losses,deck_expansion,1-2 Punch,A Fair Game,Abond the Armorsmith,Agent Hoo-man,Ammonia Clouds,Anahita the Trader,Ancient Bear,...,Z.Y.X. Researcher,Zorg,Zysysyx Shockworm,Zyzzix the Many,REDACTED,Æmber Conduction Unit,Æmber Imp,Æmberspine Mongrel,“John Smyth”,“Lion” Bautrem
count,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,...,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000,67321.000000
mean,2.520833,2.340949,373.913474,0.066294,0.025638,0.070513,0.066606,0.103593,0.007977,0.139615,...,0.084758,0.064660,0.068909,0.115061,0.010101,0.023633,0.073424,0.028253,0.123127,0.075638
std,2.452996,1.786724,44.839711,0.298569,0.160109,0.309012,0.300557,0.368056,0.089952,0.434456,...,0.341704,0.266174,0.305654,0.393665,0.103355,0.164938,0.311640,0.179880,0.411228,0.321604
min,0.000000,0.000000,341.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,341.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,341.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,3.000000,435.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,40.000000,39.000000,435.000000,4.000000,2.000000,4.000000,4.000000,4.000000,2.000000,4.000000,...,4.000000,3.000000,4.000000,4.000000,2.000000,3.000000,4.000000,3.000000,5.000000,4.000000


In [63]:
deck_df['score'] = deck_df['deck_wins'] - deck_df['deck_losses']

In [64]:
deck_df.iloc[0]['score']

1

Almost there. Now we're going to tidy up a little. We remove the columns we no longer need and we're adjusting the name on the expansion which currently is a numeric.

In [65]:
deck_df.loc[deck_df.deck_expansion == 435, 'expansion'] = True #'Call of the Archons'
deck_df.loc[deck_df.deck_expansion == 341, 'expansion'] = False #'Age of Ascension'

In [66]:
deck_df = deck_df.drop("deck_expansion", axis=1)
deck_df = deck_df.drop("deck_list", axis=1)
deck_df = deck_df.drop("card_name_list", axis=1)

In [67]:
deck_df.loc[0]

deck_id                    6216e565-a790-4e5b-b89a-4a814917b94e
deck_name                Leif “As”, Granjera de la Alcantarilla
deck_wins                                                     2
deck_losses                                                   1
house_brobnar                                              True
house_dis                                                  True
house_sanctum                                              True
house_mars                                                False
house_untamed                                             False
house_shadows                                             False
house_logos                                               False
1-2 Punch                                                     0
A Fair Game                                                   0
Abond the Armorsmith                                          0
Agent Hoo-man                                                 0
Ammonia Clouds                          

In [68]:
deck_df.to_csv("data/data_decks_cleaned.csv", header=True, index=False)

Data prep ends here

------------------------------------------------------

